In [ ]:
# !pip install -q pytorch-lightning
!pip install -q --upgrade transformers datasets rouge_score
!pip install accelerate -U
!pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incomp

In [ ]:
!nvidia-smi

Thu Jun 13 23:46:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   40C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
transformers.__version__

NameError: name 'transformers' is not defined

In [ ]:
import transformers
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import pandas as pd
import numpy as np

import torch.nn.functional as F
# import pytorch_lightning as pl
import torch
# from pytorch_lightning.callbacks import ModelCheckpoint

import math
import random
import re
import argparse
import datetime

import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
from IPython.display import display, HTML
import random
import nltk  # Here to have a nice missing dependency error message early on
import numpy as np
from datasets import load_dataset, load_metric

import transformers
from filelock import FileLock
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed,
)
from transformers.file_utils import is_offline_mode
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version

nltk.download('punkt')

# import wandb
# wandb.login()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### References: [Official](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/summarization.ipynb#scrollTo=vc0BSBLIIrJQ), [Community](https://colab.research.google.com/github/elsanns/xai-nlp-notebooks/blob/master/fine_tune_bart_summarization_two_langs.ipynb#scrollTo=6R9d7ELIpX9F)

In [ ]:
BASE_PATH = "/content/drive/My Drive/"

MODEL_PATH = "google/flan-t5-large"
TOKENIZER_PATH = "google/flan-t5-large"
DATASET_BASE_PATH = BASE_PATH + "StrategyQA/"
DATASET_TRAIN_PATH = BASE_PATH + "strategyqa-train-with-multiple-types.jsonl"
DATASET_TEST_PATH = DATASET_BASE_PATH + "strategyqa_test.json"

SAVE_PATH = "/content/results/" + "cast_flant5_large_strategyq_with_answer/"
LOGGING_PATH = "/content/logs/" + "cast_flant5_large_strategyq_with_answer_logs/"
SAVE_MODEL_PATH = BASE_PATH + "cast_flant5_large_strategyq_with_answer:model/"

max_input_length = 128
max_target_length = 256

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# df_all = pd.read_json(DATASET_TRAIN_PATH)

# df_all = df_all.drop(["qid","term", "description", "evidence", "answer"], axis=1)
# train,val = train_test_split(df_all,test_size=0.10,random_state=42)
# raw_datasets = DatasetDict({
#     "train": Dataset.from_pandas(train),
#     "val": Dataset.from_pandas(val)
# })

print(DATASET_TRAIN_PATH)
raw_dataset = load_dataset("json", data_files=[DATASET_TRAIN_PATH])
raw_dataset_temporals = raw_dataset.filter(lambda example: 'temporal' in example["question_type"] or 'interval' in example["question_type"])
raw_datasets = raw_dataset_temporals['train'].train_test_split(test_size=0.1)

quantemp = load_dataset("json", data_files=[BASE_PATH + "train.json"])

/content/drive/My Drive/strategyqa-train-with-multiple-types.jsonl


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
quantemp_temp = quantemp.filter(lambda example: example["taxonomy_label"] == 'temporal' or example["taxonomy_label"] == 'interval')

Filter:   0%|          | 0/9935 [00:00<?, ? examples/s]

In [ ]:
quantemp_temp

DatasetDict({
    train: Dataset({
        features: ['crawled_date', 'country_of_origin', 'label', 'url', 'lang', 'claim', 'doc', 'taxonomy_label', 'label_original'],
        num_rows: 4213
    })
})

In [ ]:
# # data_subq["subquestions"] = data_subq["subquestions"].apply(lambda x: x.replace("[","").replace("]",""))
# # data_subq

DatasetDict({
    train: Dataset({
        features: ['question', 'facts', 'decomposition', 'question_type'],
        num_rows: 2061
    })
    test: Dataset({
        features: ['question', 'facts', 'decomposition', 'question_type'],
        num_rows: 229
    })
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
def prepreprocess_function(example):
  example["subquestions"] = ''.join([f"Question {i}: {example['decomposition'][i]}\n" for i in range(len(example['decomposition']))])
  return example

In [ ]:
def preprocess_function(examples, prefix="decompose the compositional question:"):
    inputs = [prefix + doc for doc in examples["question"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    labels = tokenizer(text_target=examples["subquestions"], max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
raw_datasets = raw_datasets.map(prepreprocess_function)
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True, remove_columns=raw_datasets['train'].column_names)

Map:   0%|          | 0/591 [00:00<?, ? examples/s]

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

Map:   0%|          | 0/591 [00:00<?, ? examples/s]

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

In [ ]:
# raw_quantemp_temp = quantemp_temp.map(prepreprocess_function)

In [ ]:
# train['question'][0]
# raw_datasets['train']['question_type']

[['temporal', 'comparison'],
 ['temporal', 'comparison'],
 ['temporal', 'comparison'],
 ['temporal', 'comparison'],
 ['temporal'],
 ['temporal', 'comparison'],
 ['temporal', 'comparison'],
 ['statistical', 'temporal'],
 ['temporal', 'comparison'],
 ['temporal', 'comparison'],
 ['statistical', 'temporal'],
 ['interval'],
 ['statistical', 'temporal'],
 ['temporal'],
 ['temporal', 'comparison'],
 ['temporal'],
 ['temporal'],
 ['temporal', 'comparison'],
 ['temporal', 'comparison'],
 ['temporal'],
 ['temporal'],
 ['interval', 'comparison'],
 ['statistical', 'temporal'],
 ['temporal', 'statistical', 'comparison'],
 ['temporal', 'comparison'],
 ['temporal'],
 ['statistical', 'temporal'],
 ['temporal', 'comparison'],
 ['temporal'],
 ['temporal', 'comparison'],
 ['interval', 'temporal'],
 ['interval', 'comparison'],
 ['temporal'],
 ['temporal', 'comparison'],
 ['temporal', 'comparison'],
 ['statistical', 'temporal'],
 ['temporal', 'comparison'],
 ['temporal'],
 ['temporal', 'comparison'],
 ['t

In [ ]:
tokenized_datasets["train"]["input_ids"][2]

[20,
 287,
 2748,
 15,
 8,
 5761,
 138,
 822,
 10,
 308,
 23,
 26,
 8,
 1687,
 13,
 12249,
 23967,
 43,
 3,
 9,
 1516,
 1504,
 30,
 707,
 5,
 679,
 302,
 7,
 58,
 1]

In [ ]:
raw_datasets["train"]

Dataset({
    features: ['question', 'facts', 'decomposition', '__index_level_0__', 'subquestions'],
    num_rows: 2061
})

In [ ]:
# raw_train = Dataset.from_pandas(train)
# val = val.drop(["qid","term", "description", "evidence", "answer"], axis=1)
# print(val)
raw_val = Dataset.from_pandas(val)

In [ ]:
train.to_csv("train_strategyq_with_answer.csv",index=False)
val.to_csv("val_strategyq_with_answer.csv",index=False)

# Loading and processing the data

In [ ]:
TRAIN_FILE_PATH = "train_strategyq_with_answer.csv"
VAL_FILE_PATH = "val_strategyq_with_answer.csv"

In [ ]:
data_files = {}
data_files["train"] = TRAIN_FILE_PATH
data_files["validation"] = VAL_FILE_PATH

In [ ]:
xmetric = load_metric("rouge")
data_files

<ipython-input-38-23cc36f6b141>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  xmetric = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
metric = load_metric("rouge", trust_remote_code=True)


<ipython-input-9-33986b74e8a2>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge", trust_remote_code=True)


In [ ]:
raw_datasets["train"][0]

{'question': 'Did Pink Floyd have a song about the French Riviera?',
 'facts': ['Pink Floyd included the song San Tropez on the album Meddle',
  'San Tropez, also known as Saint Tropez, is a town located on the French Riviera'],
 'decomposition': ['What songs by Pink Floyd are named after towns in France?',
  'What are the names of the major towns in the French Riviera?',
  'Are any towns in #1 also in #2?'],
 '__index_level_0__': 1056,
 'subquestions': ['Question 0: What songs by Pink Floyd are named after towns in France?\n',
  'Question 1: What are the names of the major towns in the French Riviera?\n',
  'Question 2: Are any towns in #1 also in #2?\n']}

In [ ]:
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["train"])

,question,facts,decomposition,__index_level_0__,subquestions
0,Is a doctorate required to teach at a SUNY School?,"[At SUNY schools, there are some full time professors with doctorates., At SUNY schools, there are adjunct professors who teach with a Masters degree. ]","[Is it the case, that there are no people teaching at SUNY that do not have a doctorate degree?]",1645,"[Question 0: Is it the case, that there are no people teaching at SUNY that do not have a doctorate degree?\n]"
1,Is a cory catfish likely to eat another living fish?,"[The cory catfish is a fish that is described as a bottom feeder., The cory catfish feeds on food located at the bottom of an ocean., Fish cannot live too deep in oceans for very long because of the intense water pressure., The bottom of oceans is populated by algae, coral, and microorganisms.]","[What do cory catfish eat?, Is fish part of #1?]",2285,"[Question 0: What do cory catfish eat?\n, Question 1: Is fish part of #1?\n]"
2,Does Neville Longbottom have more courage as a child than as an adult?,"[Neville Longbottom is a character from the Harry Potter series., In the first few books of the Harry Potter series, Neville is a child., In the final few books of the Harry Potter series Neville is becoming an adult. , Neville's first appearances in the series show him to be very cowardly., Neville is considered a hero by the end of the series.]","[Did Neville Longbottom's first appearances in the series show him to be very cowardly?, Was #1's a child in the first few books of the Harry potter series?, Was Neville Longbottom considered a hero by the end of the series?, Was #3's an adult in the final few books of the Harry potter series?, Was he more courageous in #2 than #4?]",257,"[Question 0: Did Neville Longbottom's first appearances in the series show him to be very cowardly?\n, Question 1: Was #1's a child in the first few books of the Harry potter series?\n, Question 2: Was Neville Longbottom considered a hero by the end of the series?\n, Question 3: Was #3's an adult in the final few books of the Harry potter series?\n, Question 4: Was he more courageous in #2 than #4?\n]"
3,Did Metallica band members cutting their hair hurt their sales?,"[Metallica famously cut their hair in 1996 which caused a huge divide in their fanbase., Metallica's best selling album, The Black Album, was released in 1991 and has sold over 20 million copies., Since 1996, Metallica have released 5 studio albums., Metalica's 5 studio albums since 1996 have sold around a combined 14 million copies]","[When did Metallica band members cut their hair?, How many copies of their best selling album has been sold?, How many copies of their last five albums have been sold altogether?, Is #1 after the release date of #2 and before those of #3, and #2 greater than #3?]",2128,"[Question 0: When did Metallica band members cut their hair?\n, Question 1: How many copies of their best selling album has been sold?\n, Question 2: How many copies of their last five albums have been sold altogether?\n, Question 3: Is #1 after the release date of #2 and before those of #3, and #2 greater than #3?\n]"
4,Is xenophobia hypothetically unimportant between Saladin and Ali Askari?,"[Xenophobia is the dislike of someone that is foreign or from a different background., Saladin was a Kurdish leader that became sultan of Egypt., Ali Askari was a Kurdish politician.]","[Which relation between two parties could lead bring about xenophobia?, What was Saladin's ethnicity?, What was Ali Askari's ethnicity?, Does the relation between #2 and #3 fail to describe #1?]",1398,"[Question 0: Which relation between two parties could lead bring about xenophobia?\n, Question 1: What was Saladin's ethnicity?\n, Question 2: What was Ali Askari's ethnicity?\n, Question 3: Does the relation between #2 and #3 fail to describe #1?\n]"


## Tokenization and Dataset Preparation

In [ ]:
tokenizer

T5TokenizerFast(name_or_path='google/flan-t5-large', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>

In [ ]:
def preprocess_function(examples, prefix="decompose the compositional question:"):
    facts =
    inputs = [prefix + examples["question"][doc] + for doc in range(len(examples["question"]))]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # The "labels" are the tokenized outputs:
    print(examples["subquestions"])
    labels = tokenizer(text_target=examples["subquestions"], max_length=max_target_length, truncation=True)
    print(labels)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True, remove_columns=raw_datasets['train'].column_names)

Map:   0%|          | 0/2061 [00:00<?, ? examples/s]

['Question 0: What songs by Pink Floyd are named after towns in France?\nAnswer 0: Pink Floyd included the song San Tropez on the album MeddleQuestion 1: What are the names of the major towns in the French Riviera?Question 2: Are any towns in #1 also in #2?', 'Question 0: Where does Moon Jae-in live?\nAnswer 0: The President of South Korea lives in the Blue HouseQuestion 1: When was the construction of #1 finished?\nAnswer 1: The Blue House finished construction in early 1991Question 2: When was the construction of the World Trade Center completed?Question 3: Is #2 before #3?', "Question 0: What relatives did Helen Palmer have when she died?\nAnswer 0: Dr. Seuss's real name was Theodor Geisel.Question 1: What is Dr. Suess's real name?\nAnswer 1: Theodor Geisel was married to Helen Palmer at the time of her suicide.Question 2: Is #2 one of #1?", 'Question 0: What are some common dishes served during Thanksgiving?\nAnswer 0: Menthol is the main component of peppermint oil and is responsi

Map:   0%|          | 0/229 [00:00<?, ? examples/s]

['Question 0: What is the behavior and social nature of an ocelot?\nAnswer 0: An ocelot is a small wild cat native to South America, Mexico, and the southern US.Question 1: Is #1 suitable for kindergartners considering their age?\nAnswer 1: Ocelots are carnivores that hunt other animals and are aggressive, and strong for their size.', 'Question 0: When did Stone Cold Steve Austin start wrestling?\nAnswer 0: A century is a period of 100 years.Question 1: When did Stone Cold Steve Austin stop wrestling?\nAnswer 1: Stone Cold Steve Austin made his wrestling debut on September 30, 1989.Question 2: In what century is #1?\nAnswer 2: Stone Cold Steve Austin retired on March 30, 2003.Question 3: In what century is #2?\nAnswer 3: The 20th (twentieth) century was a century that began on January 1, 1901 and ended on December 31, 2000.Question 4: Is #4 minus #3 greater than 1?', 'Question 0: When were the Beatles active as a full group?\nAnswer 0: The Beatles were active from 1960 until 1969.Quest

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2061
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 229
    })
})

# Building the model

## Metrics

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    print(predictions[0])
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    print("decoded gen",decoded_preds)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

## Loading the model

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

## Training Args

In [ ]:
epochs = 10
batch_size = 8
lr = 5e-5

In [ ]:
args = Seq2SeqTrainingArguments(
    output_dir=SAVE_PATH,
    learning_rate=lr,
    do_train = True,
    do_eval = True,
    evaluation_strategy="epoch",
    save_strategy='epoch',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    load_best_model_at_end=True,
    num_train_epochs=epochs,
    predict_with_generate=True,
    generation_max_length=512,
    logging_dir=LOGGING_PATH,
    logging_steps=300,
    # report_to = "wandb",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
wandb_run = wandb.init(
    project="flant5_subq_temporal",
    config={
        "per_device_train_batch_size": batch_size,
        "learning_rate": lr})

now = datetime.datetime.now()
current_time = now.strftime("%m/%d/%Y, %H:%M:%S")
wandb_run.name = "run_" + current_time

NameError: name 'wandb' is not defined

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

In [ ]:
# tokenized_datasets["val"]['input_ids'][3]
print(raw_datasets["test"]["question"])

['Is The Joy of Painting TV show still producing new episodes?', 'Does American Independence Day occur during autumn?', 'Could you watch all of JAG in six months?', 'Can you give at least one word from the Torah to all residents of Bunkie Louisiana?', 'Were there fifty English kings throughout the Middle Ages?', 'Has categories of Nobel prizes remained same since Alfred Nobel established them?', 'Were muskets used in the Pacific War?', 'Would an ancient visitor to Persia probably consume crocus threads?', 'Is 500GB USB device enough to save 10 hours of Netflix shows a day?', 'Are any of the destinations of Japan Airlines former Axis Powers?', 'Is Edward Snowden in hiding from the United States?', 'Is Chinese successor to Chevrolet Cruze name a town far from Milan?', "Is ABBA's 1970's genre still relevant today?", 'Does the country that received the most gold medals during the 1976 Olympics still exist?', 'Is Cambodia too expensive for 2020 richest billionaire to own?', 'Were the first 

# Training

In [ ]:
trainer.evaluate() # Evaluation before fine-tuning

[    0    37 14988    13 19992  1424   504    19     3     9  4390   504
    24   799     7    30     8  7547  1229     5     1  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


{'eval_loss': 2.420825719833374,
 'eval_rouge1': 18.3794,
 'eval_rouge2': 7.0962,
 'eval_rougeL': 14.9731,
 'eval_rougeLsum': 16.7958,
 'eval_gen_len': 23.8333,
 'eval_runtime': 32.4903,
 'eval_samples_per_second': 2.031,
 'eval_steps_per_second': 0.277}

In [ ]:
val

,question,answer,subquestions
2178,Is an ocelot a good present for a kindergartener?,False,Question 0: What is the behavior and social na...
1271,Did Stone Cold Steve Austin wrestle in three d...,False,Question 0: When did Stone Cold Steve Austin s...
1268,Did the Beatles write any music in the Disco g...,False,Question 0: When were the Beatles active as a ...
44,If your skin was turning the color of a zombie...,True,Question 0: What color skin are zombies typica...
2025,Did Queen Elizabeth The Queen Mother and her d...,True,Question 0: Which name did the Queen Mother an...
...,...,...,...
886,Do tourists prefer Tuvalu to Niue?,False,Question 0: What is the average number of tour...
1742,Is hanging a viable execution method on a ship...,True,Question 0: What materials are necessary for h...
1755,Will the producer of Easy Rider become an octo...,False,Question 0: Who produced Easy Rider?\nAnswer 0...
1750,Can the Department of Defense perform a solo?,False,Question 0: How many are part of a solo perfor...


In [ ]:
# %%wandb

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.976967,58.980600,34.569500,49.724500,56.876000,39.651500
2,No log,0.960464,59.429700,35.599200,49.488600,57.424900,39.348500
3,No log,0.958055,59.888600,35.229300,49.080000,58.012200,37.939400
4,No log,0.957458,60.503400,37.082700,50.371800,58.722500,39.075800
5,0.979400,0.973375,60.383900,35.310000,48.833300,58.142500,38.651500
6,0.979400,0.982954,59.782100,35.291700,49.311600,57.624400,39.954500
7,0.979400,1.002750,60.419200,35.623500,49.336100,58.210700,39.060600
8,0.979400,1.010815,60.048000,35.403300,49.333500,57.849400,39.242400
9,0.664700,1.027024,60.263400,35.822900,48.771000,58.069300,38.621200
10,0.664700,1.028866,59.887300,35.421900,49.042100,57.741900,39.939400


[    0 11860     3   632    10   366    47     8  3504   109    51 21616
    58 11860   209    10   366    47   901  4000   782  2170    58 11860
   204    10    27     7  7172   274 15493    58     3     1     0     0
     0     0     0     0     0     0     0  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100]
decoded gen ['Question 0: When was the Harlem Renaissance? Question 1: When was Al Capone born? Question 2: Is #1 before #2? ', "Question 0: What was Alfred Hitchcock's first film? Question 1: What was the internet slang called? Question 2: Is #1 the same as #2? ", "Question 0: What is Henry Ford's last name? Question 1: What is the most common name in the United States? Question 2: Is #1 the same as #2? ", 'Question 0: How long does it take for a human heart to beat? Question 1: How long does a Toyota Hilux travel? Question 2: Is #2 greater than #1? ', 'Question 0: What is the primary material used in US 2020 nickels? Questio

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=740, training_loss=0.7782541017274599, metrics={'train_runtime': 2375.4673, 'train_samples_per_second': 2.488, 'train_steps_per_second': 0.312, 'total_flos': 850229797367808.0, 'train_loss': 0.7782541017274599, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

[    0 11860     3   632    10   366   410     8  3504   109    51 21616
  1731    58 11860   209    10   366   410   901  4000   782    67    58
 11860   204    10    27     7 15493   274  7172    58     3     1     0
     0     0     0     0     0     0     0     0  -100  -100  -100  -100
  -100]
decoded gen ['Question 0: When did the Harlem Renaissance begin? Question 1: When did Al Capone die? Question 2: Is #2 before #1? ', 'Question 0: When did Alfred Hitchcock die? Question 1: When did internet slang become common? Question 2: Is #1 before #2? ', "Question 0: What was Henry Ford's last name? Question 1: What is the most common last name in the United States? Question 2: Is #1 the same as #2? ", 'Question 0: How long is the distance between NYC and Raleigh, NC? Question 1: How long is the average Toyota Hilux? Question 2: Is #1 greater than or equal to #2? ', 'Question 0: What is the primary material used in US 2020 nickels? Question 1: What is the primary material used in nickel

{'eval_loss': 0.9574577808380127,
 'eval_rouge1': 60.5034,
 'eval_rouge2': 37.0827,
 'eval_rougeL': 50.3718,
 'eval_rougeLsum': 58.7225,
 'eval_gen_len': 39.0758,
 'eval_runtime': 17.5845,
 'eval_samples_per_second': 3.753,
 'eval_steps_per_second': 0.512,
 'epoch': 10.0}

In [ ]:
trainer.save_model("/content/drive/My Drive/flant5_strategyq_temporal")

In [ ]:
wandb_run.finish()

NameError: name 'wandb_run' is not defined

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(BASE_PATH + "flant5_strategyq_temporal")

# Evaluation

In [ ]:
quantemp_temp['train']['claim'][0:10]

['Florida residents affected by Hurricane Irma can receive $197 in food stamp benefits, but only if they can show that their homes lost power for more than two hours.',
 'Violence video shows rival groups fighting at 2022 football tournament in Democratic Republic of Congo, not March 2023 protests in Kenya',
 'In Virginia, "people are going and getting marriage licenses who are 12 and 13, and 14 and 15 years old."',
 'Minnesota expected to get 16 inches of snowfall within the next week!',
 'The video shows G20 delegates having lunch with their hands',
 'Honeybees were not considered “endangered” in the U.S. as of September 2023.',
 "The Domino's pizza chain is offering coupons for two free large pizzas to social media users who fill out an online survey.",
 'Does This Video Show Moldova Residents Blocking A NATO Column In 2022?',
 'Photo shows Imran Khan supporter waving gun during rally in 2023',
 'A patent for coronavirus was granted in 2018 to the Pirbright Institute UK, founded by 

In [ ]:
# def generate_output(test_samples, model):
#     inputs = tokenizer(
#         test_samples,
#         max_length=64,
#         return_tensors="pt")
#     model.to('cuda')
#     input_ids = inputs.input_ids.to(model.device)
#     # print(input_ids)
#     # print(tokenizer.decode(input_ids[0], skip_special_tokens=True))
#     attention_mask = inputs.attention_mask.to(model.device)
#     outputs = model.generate(input_ids, attention_mask=attention_mask, max_length = 64, do_sample=True, top_p=0.95, top_k=50)
#     output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
#     print(output_str)
#     return output_str

def generate_output(test_samples, model):
    inputs = tokenizer(
        test_samples,
        max_length=128,
        return_tensors="pt")

    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask,min_length = 64, max_length = 128, do_sample=True, top_p=0.95, top_k=50)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    print(output_str)
    return output_str

In [ ]:
prefix = "decompose the compositional question:"

['Question 0: What is the maximum duration of electricity a person must be without access to food stamps? Question 1: Is #1 less than or equal to two hours?  Question 2: Is #2 the same as #1?  answer: yes [1]. 1]. 1]. 2]. 2]. 1]. 2']


In [ ]:
# seq_list = []
# print()
# for claims in claims_batched:
#   print(claim)
#   break
#   seq_list.append(generate_output("decompose the compositional question:"+claim,model))
# len(seq_list)

seq_list = []
for claim in quantemp_temp['train']['claim'][0:1000]:
  prefix = "decompose the compositional question:"
  seq_list.append(generate_output(prefix + claim, model))
len(seq_list)

["Question 0: What is the minimum amount of time a Florida resident must show to receive food stamp benefits? Question 1: What is Hurricane Irma's maximum amount of power loss? Question 2: Is #1 greater than or equal to #2?  answer: yes..................................."]
['Question 0: When is the football tournament in Democratic Republic of Congo? Question 1: When were the March 2023 protests in Kenya? Question 2: Is #2 after #1? No Answer> Yes> No Answer> No Answer> No Answer> No Answer> No Answer> No Answer>']
['Question 0: What age groups are typically present for a marriage license in Virginia? Question 1: What age groups are most common for people going and getting marriage licenses? Question 2: What is the median age for people going and getting marriage licenses? Question 3: Is #2 greater than #3? ']
['Question 0: What is the average temperature in Minnesota currently? Question 1: How much snow is expected to fall in Minnesota in the next week? Question 2: Is #2 greater than 

1000

In [ ]:
print(seq_list[0])

["Question 0: What is the minimum amount of time a Florida resident must show to receive food stamp benefits? Question 1: What is Hurricane Irma's maximum amount of power loss? Question 2: Is #1 greater than or equal to #2?  answer: yes..................................."]


In [ ]:
subquestions = []
subquestions_retrieve = []

for item in seq_list:
  item = item[0]
  questions = re.findall(r"Question [0-2]:.*?[?]", item)
  subquestions.append(questions)
  subquestions_retrieve.append([True if re.search(r"#[0-9]", q) is None else False for q in questions])

In [ ]:
print(subquestions[5])
print(subquestions_retrieve[5])

['Question 0: In what year was the U.S. enactment of the Endangered Species Act?', 'Question 1: In what year was honeybees not considered a US state issue?', 'Question 2: Was #2 before #1?']
[True, True, False]


In [ ]:
new_dataset = quantemp_temp['train'].select(range(1000))

In [ ]:
new_dataset = new_dataset.remove_columns(["crawled_date", "country_of_origin", "url", "lang", "taxonomy_label", "label_original", "doc"])


In [ ]:
new_dataset = new_dataset.add_column("subquestions", subquestions)
new_dataset = new_dataset.add_column("subquestions_retrieve", subquestions_retrieve)

Flattening the indices:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
print(new_dataset['claim'][0])
print(new_dataset['subquestions'][0])



Florida residents affected by Hurricane Irma can receive $197 in food stamp benefits, but only if they can show that their homes lost power for more than two hours.
['Question 0: What is the minimum amount of time a Florida resident must show to receive food stamp benefits?', "Question 1: What is Hurricane Irma's maximum amount of power loss?", 'Question 2: Is #1 greater than or equal to #2?']


In [ ]:
new_dataset.to_json(BASE_PATH + "new_dataset.json")
new_dataset.to_csv(BASE_PATH + "new_dataset.csv")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

327588

In [ ]:
idx = 4
part = 'test'
print(raw_datasets)
sample = raw_datasets[part][idx]
# sample_ = tokenized_datasets[part][idx]

# print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['question', 'facts', 'decomposition', 'question_type'],
        num_rows: 591
    })
    test: Dataset({
        features: ['question', 'facts', 'decomposition', 'question_type'],
        num_rows: 66
    })
})


In [ ]:
# Checking if the tokenizer is working correctly

input_ids, labels = sample_['input_ids'], sample_['labels']
print(tokenizer.decode(input_ids, skip_special_tokens=True), tokenizer.decode(labels, skip_special_tokens=True), sep="\n")

In [ ]:
!cp -r  /content/cast_flant5_large_strategyq_with_answer:model /content/drive/MyDrive/CSAT

In [ ]:
print("decompose the compositional question:"+sample['question'], sample['decomposition'], sep="\n", end="\n\n")
generate_output("decompose the compositional question:"+sample['question'], model)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


decompose the compositional question:Can you watch the Borgia's World of Wonders before Ludacris's Release Therapy finishes?
["What is the run time of  the Borgia's World of Wonders?", "What is the run time of  Ludacris's Release Therapy?", 'Is #1 shorter than #2?']

tensor([[   20,   287,  2748,    15,     8,  5761,   138,   822,    10, 14050,
            25,  1605,     8,   272,  1677,    23,     9,    31,     7,  1150,
            13, 16347,     7,   274,  2318,    26,     9,  2685,     7,    31,
             7, 13048, 13587, 13084,    58,     1]])
decompose the compositional question:Can you watch the Borgia's World of Wonders before Ludacris's Release Therapy finishes?


["Question 0: How long is Ludacris's Release Therapy? Question 1: How long is The Borgia's World of Wonders? Question 2: Is #1 longer than #2?  Question 3: Is #1 greater than #3?   Question 4: Is #4 greater than #2? "]

In [ ]:
import pandas as pd
test_data = pd.read_csv("test_vw.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'test_vw.csv'

In [ ]:
seq_list = []
for query_paa in test_data["description"].values:
    seq_list.append(generate_output(query_paa,model))
len(seq_list)


NameError: name 'test_data' is not defined

In [ ]:
seq_list

[['Brief descriptionProvides technical services to customers.Possible tasksService deliveryEnsures that the technical service delivery process meets customer requirements. Ensures that customer needs are met. Provides technical support to the customer in the event of a technical service breakdown. Identifies and resolves technical issues and ensures that the customer understands the technical requirements.Technical service deliveryAssists in the installation, maintenance and maintenance of service delivery systems, systems, tools, and systems. Supports the installation of new systems and equipment in the area of responsibility.Customer relationship managementAssists with the implementation of customer relationship management systems. Prepares and maintains customer relationships and ensures customer satisfaction through the use of customer-centric solutions.DocumentationProvides documentation and documentation of customer needs and issues in the service delivery system. Documents custo

In [ ]:
machine_generated = [seq[0] for seq in seq_list]
machine_generated

['Brief descriptionProvides technical services to customers.Possible tasksService deliveryEnsures that the technical service delivery process meets customer requirements. Ensures that customer needs are met. Provides technical support to the customer in the event of a technical service breakdown. Identifies and resolves technical issues and ensures that the customer understands the technical requirements.Technical service deliveryAssists in the installation, maintenance and maintenance of service delivery systems, systems, tools, and systems. Supports the installation of new systems and equipment in the area of responsibility.Customer relationship managementAssists with the implementation of customer relationship management systems. Prepares and maintains customer relationships and ensures customer satisfaction through the use of customer-centric solutions.DocumentationProvides documentation and documentation of customer needs and issues in the service delivery system. Documents custom

In [ ]:
test_data = pd.read_csv("test_vw.csv")
queries_list = test_data["title"].values.tolist()

In [ ]:
final_test_df = {"queries":queries_list,"gpt_out":machine_generated, "ground_truth":list(test_data["ground_truth"].values),"description":list(test_data["description"].values),"precision":[],"recall":[],"f1":[]}

In [ ]:
actual = []
predicted = []
for target in list(test_data["ground_truth"].values):
  actual.append([target.split(" ")])

for pred in machine_generated:
  predicted.append(pred.split(" "))



In [ ]:
from nltk.translate.bleu_score import corpus_bleu

bleu_dic = {}
bleu_dic['1-grams'] = corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0))
bleu_dic['1-2-grams'] = corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0))
bleu_dic['1-3-grams'] = corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0))
bleu_dic['1-4-grams'] = corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25))

In [ ]:
bleu_dic['1-4-grams']

0.03087672955741727

In [ ]:
bleu_dic['1-3-grams']

0.06132272668423761

In [ ]:
bleu_dic['1-2-grams']

0.08624743539685949

In [ ]:
!pip install datasets
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_metric
bertscore_metric = load_metric('bertscore')

In [ ]:
bert_scores = bertscore_metric.compute(predictions=machine_generated, references=list(test_data["ground_truth"].values),model_type="distilbert-base-uncased", lang="en")
print(bert_scores)
# Normally, we use the f1-score attribute
print(bert_scores['f1'])

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loa

{'precision': [0.8293436169624329, 0.7897420525550842, 0.7427924275398254, 0.8596062064170837, 0.7655096054077148, 0.815791130065918, 0.8289499282836914, 0.7394145727157593, 0.8403562903404236, 0.7681253552436829, 0.8154471516609192, 0.8107794523239136, 0.8570922017097473, 0.7552051544189453, 0.7879809141159058, 0.7983054518699646, 0.8234920501708984, 0.8106138706207275, 0.6170855164527893, 0.824568510055542, 0.75370192527771, 0.8007907867431641, 0.8129481077194214, 0.8137281537055969, 0.7241920828819275, 0.7653471231460571, 0.7697200775146484, 0.8223090171813965, 0.8050133585929871, 0.7930941581726074, 0.844482421875, 0.7184927463531494, 0.7302052974700928, 0.793755054473877, 0.7976676225662231, 0.8294515609741211, 0.7773056030273438, 0.794824481010437, 0.8053468465805054, 0.7903791666030884, 0.82798171043396, 0.7696704864501953, 0.7847729921340942, 0.8242773413658142, 0.846941351890564, 0.7730618715286255, 0.8173378705978394, 0.8165499567985535, 0.7883261442184448, 0.834418535232544,

In [ ]:
final_test_df["precision"] = bert_scores["precision"]
final_test_df["recall"] = bert_scores["recall"]
final_test_df["f1"] = bert_scores["f1"]

In [ ]:
final_test_df = pd.DataFrame(final_test_df)
final_test_df.to_csv("bart_out_job_desc_gen.csv",index=False)

In [ ]:
from statistics import mean,median
print("f1",mean(bert_scores['f1']))
print("recall:",mean(bert_scores['recall']))
print("precision:",mean(bert_scores['precision']))

f1 0.7928438886352207
recall: 0.7927729707697163
precision: 0.7940285464991694


In [ ]:
!zip -r bart_job_desc_gen_model.zip /content/bart_job_desc_gen_model

updating: content/bart_job_desc_gen_model/ (stored 0%)
updating: content/bart_job_desc_gen_model/tokenizer_config.json (deflated 47%)
updating: content/bart_job_desc_gen_model/pytorch_model.bin (deflated 8%)
updating: content/bart_job_desc_gen_model/config.json (deflated 64%)
updating: content/bart_job_desc_gen_model/training_args.bin (deflated 48%)
updating: content/bart_job_desc_gen_model/merges.txt (deflated 53%)
updating: content/bart_job_desc_gen_model/special_tokens_map.json (deflated 52%)
updating: content/bart_job_desc_gen_model/vocab.json (deflated 59%)
updating: content/bart_job_desc_gen_model/tokenizer.json (deflated 72%)
  adding: content/bart_job_desc_gen_model/.ipynb_checkpoints/ (stored 0%)


In [ ]:
!zip -r bart_reconstruction.zip /content/bart_reconstruction

  adding: content/bart_reconstruction/ (stored 0%)
  adding: content/bart_reconstruction/checkpoint-600/ (stored 0%)
  adding: content/bart_reconstruction/checkpoint-600/tokenizer_config.json (deflated 47%)
  adding: content/bart_reconstruction/checkpoint-600/pytorch_model.bin (deflated 8%)
  adding: content/bart_reconstruction/checkpoint-600/config.json (deflated 64%)
  adding: content/bart_reconstruction/checkpoint-600/training_args.bin (deflated 48%)
  adding: content/bart_reconstruction/checkpoint-600/merges.txt (deflated 53%)
  adding: content/bart_reconstruction/checkpoint-600/rng_state.pth (deflated 28%)
  adding: content/bart_reconstruction/checkpoint-600/trainer_state.json (deflated 63%)
  adding: content/bart_reconstruction/checkpoint-600/special_tokens_map.json (deflated 52%)
  adding: content/bart_reconstruction/checkpoint-600/scheduler.pt (deflated 49%)
  adding: content/bart_reconstruction/checkpoint-600/optimizer.pt (deflated 9%)
  adding: content/bart_reconstruction/che